In [81]:
import pandas as pd
from elasticsearch import Elasticsearch
from attrdict import AttrDict
import networkx as nx
import matplotlib.pyplot as plt

# Load files

In [5]:
eval_seq = pd.read_csv('../pre_processing/resources/evaluation/2019/TREC-Competition-eval-seq-5-25000.csv', sep='\.|,', engine='python', names = ['sid','q_num','qid'])

In [29]:
es = Elasticsearch([{'timeout': 300}])

In [30]:
es

<Elasticsearch([{'timeout': 300}])>

In [24]:
eval_queries = pd.read_json('../pre_processing/resources/evaluation/2019/TREC-Competition-eval-sample-with-rel.json', lines =True)
eval_queries = eval_queries.explode('documents')
eval_queries.documents = eval_queries.documents.apply(lambda row: row.get('doc_id'))

In [25]:
eval_ids = list(set(eval_queries.documents.to_list()))
eval_ids

['...',
 ...]

In [46]:
res = es.search(index='semanticscholar2019og', body={'size':len(eval_ids),'query':{'ids':{'values':eval_ids}}})

In [47]:
ares = AttrDict(res)

In [54]:
author_id_lists = [hit['_source']['author_ids'] for hit in ares.hits.hits]

In [55]:
author_ids = list(set([author_id for author_id_list in author_id_lists for author_id in author_id_list]))

In [57]:
doc_authors_mapping = {hit['_id']:hit['_source']['author_ids'] for hit in ares.hits.hits}

In [58]:
doc_authors_mapping

{'4f5960d2bd676e8c686c0b29443cee632db1fb4d': ['38154813',
  '5528768',
  '2457100'],
 '65896a6b4b3b377c1396b8923017adb93c7097e1': ['32360997', '48297702'],
 '69da6e38e11977ecebc0cf6dc15463b4ff630b29': ['47437775',
  '2005542',
  '32669329',
  '2603525',
  '46620509',
  '4549693'],
 '9a5e7423c1cf43e47518b411b6222e1129f2ad73': ['1836532',
  '31888864',
  '2044908'],
 '6b7101e3619b85c1ea59e691d709b78db5a095f3': ['1715283', '1728757'],
 '0fc23dbc2eab3ad72853a132c99bb558d21f673b': ['1973567'],
 'ffa0f7897bd77e6106bd3fb20bd38b15d64a7b84': ['5402845',
  '34781460',
  '31493317'],
 '79b179943d788c32c66c849a0848b2187e4b5ade': ['46775628', '2668864'],
 'dd155a62460dd6754f883782f3d7c6bf552ffa47': ['14062435',
  '2040928',
  '2219411',
  '7725160'],
 'e15704ba852253cb71e8d3493b6999314696188c': ['46458853', '2989507'],
 '5496f1f6b345e75b3e41d9a9ff422e7ccf9b0c10': ['5869161', '6145082'],
 '0ea26e30e4037c493f06ec45eaef20d6b0ac0dd6': ['2668115', '2089035', '1682282'],
 '6c5337a1deea7a8588493f1ebfa01e0

In [61]:
G = nx.Graph()


In [62]:
doc_nodes = [(eval_id,{'type':'doc'}) for eval_id in eval_ids]
G.add_nodes_from(doc_nodes)


In [69]:
author_nodes = [(author_id,{'type':'author'}) for author_id in author_ids]
G.add_nodes_from(author_nodes)

In [79]:
for k,v in doc_authors_mapping.items():
    for a in v:
        G.add_edge(k,a)

In [80]:
G.edges

EdgeView([('57cfdb7cacff54c959272e199c18617d32b347bb', '3800493'), ('2d0eec270872866bb9dcc5b6ae18e85477bdac96', '5989033'), ('2d0eec270872866bb9dcc5b6ae18e85477bdac96', '2119751'), ('2d0eec270872866bb9dcc5b6ae18e85477bdac96', '2011494'), ('2d0eec270872866bb9dcc5b6ae18e85477bdac96', '1720151'), ('2d0eec270872866bb9dcc5b6ae18e85477bdac96', '33782166'), ('2d0eec270872866bb9dcc5b6ae18e85477bdac96', '39061561'), ('2a94c84383ee3de5e6211d43d16e7de387f68878', '33493200'), ('2a94c84383ee3de5e6211d43d16e7de387f68878', '3127283'), ('2a94c84383ee3de5e6211d43d16e7de387f68878', '2983898'), ('2a94c84383ee3de5e6211d43d16e7de387f68878', '39353098'), ('2a94c84383ee3de5e6211d43d16e7de387f68878', '1790580'), ('2a94c84383ee3de5e6211d43d16e7de387f68878', '1769406'), ('e78d7f6fd72e1039e7070d7d6a9509024e99fe15', '39013012'), ('e78d7f6fd72e1039e7070d7d6a9509024e99fe15', '40013375'), ('e78d7f6fd72e1039e7070d7d6a9509024e99fe15', '1776647'), ('e78d7f6fd72e1039e7070d7d6a9509024e99fe15', '1688870'), ('fbd40fbb0e960